# Live Trading Demo

This notebook runs MT5 first, then IBKR, and prints connection/runtime status to stdout.


In [1]:
import os
import sys
import asyncio
import threading
from pathlib import Path

import yaml

sys.path.append(os.path.abspath("../.."))

from nautilus_trader.config import TradingNodeConfig, LiveExecEngineConfig, LoggingConfig
from nautilus_trader.live.node import TradingNode
from nautilus_trader.model.identifiers import ClientId, InstrumentId
from trader import OneMinuteBuyHoldStrategy, OneMinuteBuyHoldConfig

accounts_path = Path("../../config/accounts.yaml").resolve()
accounts = yaml.safe_load(accounts_path.read_text(encoding="utf-8"))
print(f"Using accounts config: {accounts_path}")

project_root = accounts_path.parent.parent
log_dir = project_root / "tests" / "notebooks"
log_dir.mkdir(parents=True, exist_ok=True)

# Nautilus logging is process-global, so both runs share one runtime log file.
live_log_file_name = "nt_live_demo"
live_log_path = log_dir / f"{live_log_file_name}.log"
print(f"Shared Nautilus runtime log file: {live_log_path}")

# Per-run snapshots (copied from shared runtime log after each run).
mt5_log_path = log_dir / "nt_mt5.log"
ib_log_path = log_dir / "nt_ibkr.log"
print(f"MT5 snapshot log file: {mt5_log_path}")
print(f"IBKR snapshot log file: {ib_log_path}")

mt5_node_logging = LoggingConfig(
    log_level="INFO",
    log_level_file="INFO",
    log_directory=str(log_dir),
    log_file_name=live_log_file_name,
    clear_log_file=True,
    log_colors=False,
)

ib_node_logging = LoggingConfig(
    log_level="INFO",
    log_level_file="INFO",
    log_directory=str(log_dir),
    log_file_name=live_log_file_name,
    clear_log_file=True,
    log_colors=False,
)


async def _safe_stop_node(node: TradingNode | None, label: str) -> None:
    if node is None:
        return
    try:
        if node.is_running():
            await node.stop_async()
        deadline = asyncio.get_running_loop().time() + 15.0
        while node.is_running() and asyncio.get_running_loop().time() < deadline:
            await asyncio.sleep(0.1)
    except Exception as exc:
        print(f"{label} stop warning: {exc}")


async def _cleanup_named_node(var_name: str, label: str) -> None:
    existing = globals().get(var_name)
    if existing is None:
        return
    print(f"Cleaning existing {label} node from previous run...")
    await _safe_stop_node(existing, label)
    globals()[var_name] = None



def _reset_ibkr_factory_cache(host: str, port: int, client_id: int) -> None:
    """Clear cached IB factory singletons so reruns create a fresh IB client."""
    try:
        from nautilus_trader.adapters.interactive_brokers import factories as ib_factories
    except Exception as exc:
        print(f"IBKR factory cache import warning: {exc}")
        return

    client_keys = [
        key
        for key in list(ib_factories.IB_CLIENTS.keys())
        if len(key) >= 3 and key[0] == host and key[1] == port and key[2] == client_id
    ]
    for key in client_keys:
        client = ib_factories.IB_CLIENTS.pop(key, None)
        if client is not None:
            try:
                client.stop()
            except Exception:
                pass

    provider_keys = [
        key
        for key in list(ib_factories.IB_INSTRUMENT_PROVIDERS.keys())
        if key and key[0] == (host, port, client_id)
    ]
    for key in provider_keys:
        ib_factories.IB_INSTRUMENT_PROVIDERS.pop(key, None)

    if client_keys or provider_keys:
        print(
            f"Reset IBKR factory cache for {host}:{port} client_id={client_id} "
            f"(clients={len(client_keys)}, providers={len(provider_keys)})"
        )


Using accounts config: S:\Github\Trading\config\accounts.yaml
Shared Nautilus runtime log file: S:\Github\Trading\tests\notebooks\nt_live_demo.log
MT5 snapshot log file: S:\Github\Trading\tests\notebooks\nt_mt5.log
IBKR snapshot log file: S:\Github\Trading\tests\notebooks\nt_ibkr.log


## MT5 Run

Loads instruments into cache, starts the node in a background thread, prints health, then stops.


In [2]:
from trader.adapters.metatrader import (
    MetaTrader5DataClientConfig,
    MetaTrader5ExecClientConfig,
    MetaTrader5LiveDataClientFactory,
    MetaTrader5LiveExecClientFactory,
)

await _cleanup_named_node("mt5_node", "MT5")

mt5_cfg = accounts["venues"]["MT5"]

mt5_data_config = MetaTrader5DataClientConfig(
    mt5_login=mt5_cfg["mt5_login"],
    mt5_password=mt5_cfg["mt5_password"],
    mt5_server=mt5_cfg["mt5_server"],
    mt5_path=mt5_cfg["mt5_path"],
    bar_seconds=5,
    poll_interval=0.5,
)

mt5_exec_config = MetaTrader5ExecClientConfig(
    mt5_login=mt5_cfg["mt5_login"],
    mt5_password=mt5_cfg["mt5_password"],
    mt5_server=mt5_cfg["mt5_server"],
    mt5_path=mt5_cfg["mt5_path"],
    mt5_deviation=20,
)

mt5_strategy = OneMinuteBuyHoldStrategy(
    OneMinuteBuyHoldConfig(
        instrument_id="EURUSD.MT5",
        bar_type="EURUSD.MT5-5-SECOND-MID-EXTERNAL",
        trade_size=0.1,
        hold_seconds=5,
    )
)

mt5_node = None
try:
    mt5_node = TradingNode(
        TradingNodeConfig(
            data_clients={"MT5": mt5_data_config},
            exec_clients={"MT5": mt5_exec_config},
            exec_engine=LiveExecEngineConfig(reconciliation=False),
            logging=mt5_node_logging,
        )
    )
    mt5_node.add_data_client_factory("MT5", MetaTrader5LiveDataClientFactory)
    mt5_node.add_exec_client_factory("MT5", MetaTrader5LiveExecClientFactory)
    mt5_node.trader.add_strategy(mt5_strategy)
    mt5_node.build()

    # Manually load instruments into cache before strategy start.
    mt5_dc = mt5_node.kernel.data_engine._clients[ClientId("MT5")]
    mt5_provider = mt5_dc._instrument_provider
    await mt5_provider.load_all_async()
    for inst in mt5_provider._instruments.values():
        mt5_node.cache.add_instrument(inst)

    cached = mt5_node.cache.instrument(InstrumentId.from_str("EURUSD.MT5"))
    print("MT5 instrument cached:", cached is not None)

    mt5_node.run()

    for i in range(5):
        await asyncio.sleep(5)
        print(
            f"MT5 t={5 * (i + 1)}s connected data={mt5_node.kernel.data_engine.check_connected()} exec={mt5_node.kernel.exec_engine.check_connected()}"
        )
finally:
    await _safe_stop_node(mt5_node, "MT5")
    if live_log_path.exists():
        mt5_log_path.write_bytes(live_log_path.read_bytes())
        print(f"MT5 run complete. Snapshot saved to: {mt5_log_path}")


MT5 instrument cached: True
MT5 t=5s connected data=True exec=True
MT5 t=10s connected data=True exec=True
MT5 t=15s connected data=True exec=True
MT5 t=20s connected data=True exec=True
MT5 t=25s connected data=True exec=True
MT5 run complete. Snapshot saved to: S:\Github\Trading\tests\notebooks\nt_mt5.log


## IBKR Run

Attempts a live IBKR run and prints status/errors to stdout.


In [2]:
from nautilus_trader.adapters.interactive_brokers.factories import (
    InteractiveBrokersLiveDataClientFactory,
    InteractiveBrokersLiveExecClientFactory,
)
from ibapi.common import MarketDataTypeEnum as IBMarketDataTypeEnum
from nautilus_trader.adapters.interactive_brokers.config import SymbologyMethod
from trader.adapters.ibkr import (
    ibkr_data_config,
    ibkr_exec_config,
    ibkr_instrument_config,
)

await _cleanup_named_node("ib_node", "IBKR")

ib_cfg = accounts["venues"].get("IDEALPRO", {})
ib_host = ib_cfg.get("host", "127.0.0.1")
ib_port = int(ib_cfg.get("port", 7497))
ib_client_id = int(ib_cfg.get("client_id", 1))
ib_account = ib_cfg.get("account", "")

_reset_ibkr_factory_cache(ib_host, ib_port, ib_client_id)

ib_instr_cfg = ibkr_instrument_config(
    load_ids=frozenset({InstrumentId.from_str("EUR/USD.IDEALPRO")}),
    symbology_method=SymbologyMethod.IB_SIMPLIFIED,
)

ib_data = ibkr_data_config(
    host=ib_host,
    port=ib_port,
    client_id=ib_client_id,
    market_data_type=IBMarketDataTypeEnum.DELAYED_FROZEN,
    use_regular_trading_hours=False,
    instrument_provider=ib_instr_cfg,
    routing_venues=["IDEALPRO"],
)
ib_exec = ibkr_exec_config(
    host=ib_host,
    port=ib_port,
    client_id=ib_client_id,
    account=ib_account,
    instrument_provider=ib_instr_cfg,
    routing_venues=["IDEALPRO"],
)

# IB FX size is base-currency units (EUR for EUR/USD), not lots.
ib_strategy = OneMinuteBuyHoldStrategy(
    OneMinuteBuyHoldConfig(
        instrument_id="EUR/USD.IDEALPRO",
        bar_type="EUR/USD.IDEALPRO-15-SECOND-MID-EXTERNAL",
        trade_size=10000.0,
        hold_seconds=5,
        time_in_force="DAY",
        exec_client_id="IDEALPRO",
    )
)

ib_node = None
try:
    ib_node = TradingNode(
        TradingNodeConfig(
            data_clients={"IDEALPRO": ib_data},
            exec_clients={"IDEALPRO": ib_exec},
            exec_engine=LiveExecEngineConfig(reconciliation=False),
            logging=ib_node_logging,
        )
    )
    ib_node.add_data_client_factory("IDEALPRO", InteractiveBrokersLiveDataClientFactory)
    ib_node.add_exec_client_factory("IDEALPRO", InteractiveBrokersLiveExecClientFactory)
    ib_node.trader.add_strategy(ib_strategy)
    ib_node.build()

    ib_node.run()

    for i in range(12):
        await asyncio.sleep(5)
        print(
            f"IBKR t={5 * (i + 1)}s connected data={ib_node.kernel.data_engine.check_connected()} exec={ib_node.kernel.exec_engine.check_connected()}"
        )
finally:
    await _safe_stop_node(ib_node, "IBKR")
    if live_log_path.exists():
        ib_log_path.write_bytes(live_log_path.read_bytes())

ib_log_text = ib_log_path.read_text(encoding="utf-8", errors="ignore") if ib_log_path.exists() else ""
print(f"IBKR submit count: {ib_log_text.count('SubmitOrder')}")
print(f"IBKR submitted events: {ib_log_text.count('OrderSubmitted')}")
print(f"IBKR accepted events: {ib_log_text.count('OrderAccepted')}")
print(f"IBKR filled events: {ib_log_text.count('OrderFilled')}")
print(f"IBKR canceled events: {ib_log_text.count('OrderCanceled')}")
print(f"IBKR expired events: {ib_log_text.count('OrderExpired')}")
print(f"IBKR rejected events: {ib_log_text.count('OrderRejected')}")
print(f"IBKR denied events: {ib_log_text.count('OrderDenied')}")
if "no execution client configured for IDEALPRO" in ib_log_text:
    print("IBKR routing error: no IDEALPRO execution client route.")
if "code: 420" in ib_log_text:
    print("IBKR market-data entitlement error (420). Enable US equity market data or switch instrument.")
print(f"IBKR run complete. Snapshot saved to: {ib_log_path}")


IBKR t=5s connected data=True exec=True
IBKR t=10s connected data=True exec=True
IBKR t=15s connected data=True exec=True
IBKR t=20s connected data=True exec=True
IBKR t=25s connected data=True exec=True
IBKR t=30s connected data=True exec=True
IBKR submit count: 1
IBKR submitted events: 1
IBKR accepted events: 0
IBKR filled events: 0
IBKR canceled events: 0
IBKR expired events: 0
IBKR rejected events: 0
IBKR denied events: 0
IBKR run complete. Snapshot saved to: S:\Github\Trading\tests\notebooks\nt_ibkr.log
